In [131]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from torchvision.models import detection
import torchvision
from tqdm import tqdm, tqdm_notebook
from PIL import Image
import matplotlib

In [102]:
!unzip -q "drive/MyDrive/Colab Notebooks/Тестовые снимки.zip"

In [ ]:
 def get_cropped_image(picture, coordinates, expand_coeff=2.0):
        '''
            picture: torch.Tensor
            coordinates: "indexed set" of 4 coordinates: [x1, y1, x2, y2]
        '''
        x = coordinates[0]
        y = coordinates[1]
        w = coordinates[2] - coordinates[0]
        h = coordinates[3] - coordinates[1]
        w_expanded = int(w * expand_coeff)
        h_expanded = int(h * expand_coeff)
        x_expanded = int(x - (w_expanded - w) / 2.0)
        y_expanded = int(y - (h_expanded - h) / 2.0)

        return torchvision.transforms.functional.crop(
            picture, 
            int(y_expanded), 
            int(x_expanded), 
            int(h_expanded), 
            int(w_expanded),
        )

In [ ]:
# picture: torch.Tensor or np.array
# boxes: indexed object
# labels: indexed object
def draw_pic_with_rect(picture, boxes, labels, n=5):
    boxes = boxes[:n]
    labels = labels[:n]
    if isinstance(picture, torch.Tensor):
        picture = (picture.detach().squeeze(0) * 256).permute(1, 2, 0).numpy()
    picture = picture.astype(dtype=np.int)

    fig, ax = plt.subplots(figsize = (15, 15))
    ax.imshow(picture)

    for box, lab in zip(boxes, labels):
        rect = matplotlib.patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                       linewidth=1, edgecolor='r', facecolor='none')
        ax.text(box[0], box[1], inst_classes[lab], fontsize = 12)
        ax.add_patch(rect)

    fig.show()

In [2]:
inst_classes = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [339]:
animals = [
            "an animal", 
            "not an animal",
            "a person",
            "a bench",
            "a car",
]

is_dog = [
            "a dog",
            "a cat",
            "a bird",
            "a bench",
]

interensting_indices = [
    inst_classes.index("bird"),
    inst_classes.index("cat"),
    inst_classes.index("dog"),
    inst_classes.index("horse"),
    inst_classes.index("sheep"),
    inst_classes.index("cow"),
    inst_classes.index("elephant"),
    inst_classes.index("bear"),
    inst_classes.index("zebra"),
    inst_classes.index("giraffe"),
    inst_classes.index("bench"),          
    inst_classes.index("person"),
]

In [17]:
converter_tail = {
    "a dachshund": 1,
    "a borzoi" : 2,
    "a basset hound" : 2,
    "a spitz": 1,
    "a yorkshire terrier": 1,
    "a chihuahua": 1,
    "a german shepherd" : 2,
    "a labrador" : 2,
    "a husky" : 2,
    "a jack russell terrier" : 2,
    "a caucasian shepherd dog": 1,
    "a corgi": 1,
    "a bulldog": 1,
    "a pug": 1,
    "a poodle": 1,
    "a golden retriever" : 2,
    "a rottweiler": 1,
    "a beagle" : 2,
    "a cocker spaniel": 1,
    "a deutscher boxer": 1,
    "a pekingese": 1,
    "a sharpey": 1,
    "a bull terrier" : 2,
    "a doberman": 1,
    "a st. bernard" : 2,
    "a newfoundland" : 2,
    "a border collie" : 2,
    "a shiba inu": 1,
    "a dalmatian" : 2,
    "a samoyed" : 2,
    "a bichon frise": 1,
    "a toy terrier": 1,
    "a cavalier king charles spaniel": 2,
}

In [3]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ug3qbliy
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-ug3qbliy
     |████████████████████████████████| 64 kB 2.4 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369107 sha256=b9f356b6cf7e94432c318fcbae2f064653b441d73258380d4d48f039d6aa93dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-agf4b1qp/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=b650adf6c6f4fd38b7a93d8e9886b1d724d763f2a695c367639e77740f81a2a7
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built clip ftfy


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [25]:
model_detection = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained = True).to(device=device)

In [10]:
import clip
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 117MiB/s]


In [11]:
!pip3 install pytesseract
!sudo apt install tesseract-ocr
!sudo apt-get install tesseract-ocr-rus
!pip3 install opencv-contrib-python --upgrade

  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14072 sha256=b748cf2bca461257f8510086f8b8d5e59ce098ffecf377571a10688bf0467033
  Stored in directory: /root/.cache/pip/wheels/a4/89/b9/3f11250225d0f90e5454fcc30fd1b7208db226850715aa9ace
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http:/

In [13]:
import pytesseract
import cv2
import re

In [18]:
converter_color = {
    "a black dog": 1,
    "a white dog": 2,
    "a brown dog": 3,
    "an orange dog": 3,
    "a chocolate dog": 3,
    "a liver dog": 3,
    "a red dog": 3,
    "a gold dog": 2,
    "a cream dog": 2,
    "a fawn dog": 2,
    "a blue dog": 3,
    "a gray dog": 2,
    "a grey dog": 2,
}

In [130]:
# picture: torch.Tensor or np.array
# boxes: indexed object
# labels: indexed object
def draw_pic_with_rect(picture, boxes, labels, n=5):
    boxes = boxes[:n]
    labels = labels[:n]
    if isinstance(picture, torch.Tensor):
        picture = (picture.cpu().detach().squeeze(0) * 256).permute(1, 2, 0).numpy()
    picture = picture.astype(dtype=np.int)

    fig, ax = plt.subplots(figsize = (15, 15))
    ax.imshow(picture)

    for box, lab in zip(boxes, labels):
        rect = matplotlib.patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                       linewidth=1, edgecolor='r', facecolor='none')
        ax.text(box[0], box[1], inst_classes[lab], fontsize = 12)
        ax.add_patch(rect)

    fig.show()

In [295]:
class ResultType:
    def __init__(self, 
                 filename, 
                 is_animal_there, 
                 is_it_a_dog, 
                 is_the_owner_there, 
                 color, 
                 breed,  
                 tail, 
                 address, 
                 cam_id,  
                 date):
        
        self.res = {
            "filename": filename,
            "is_animal_there": is_animal_there,
            "is_it_a_dog": is_it_a_dog,
            "is_the_owner_there": is_the_owner_there,
            "color": color,
            "breed": breed,
            "tail": tail,
            "address": address,
            "cam_id": cam_id,
            "date": date,
        }

    def make_readable(self):
        raise NotImplementedError()
    


In [343]:
class FindLostAnimal():
    def __init__(self, 
                 model_detection, 
                 model_clip, 
                 preprocess,
                 model_superres,
                 interesting_indices,
                 animals, 
                 is_animal_index,
                 is_dog,
                 is_dog_index,
                 breeds,
                 colors,
                 converter_tail,
                 converter_color,
                 person_label=1,
                 device="cpu"
                 ):
        
        self.device = device
        self.model_detection = model_detection
        self.model_clip = model_clip
        self.preprocess_clip = preprocess
        self.model_superres = model_superres # TODO

        self.interesting_indices = interesting_indices

        self.tokens_animal = clip.tokenize(animals).to(self.device)
        self.tokens_color = clip.tokenize(colors).to(self.device)
        self.tokens_is_dog = clip.tokenize(is_dog).to(self.device)
        self.tokens_breed = clip.tokenize(breeds).to(self.device)
        self.is_animal_index = is_animal_index
        self.is_dog_index = is_dog_index
        self.breeds = breeds
        self.colors = colors
        self.converter_tail = converter_tail
        self.converter_color = converter_color
        self.person_label = person_label

        self.model_detection.eval()
        self.model_clip.eval()

        self.num_saved_pictures = 0 # TODO

    def __get_cropped_image(self, picture: torch.Tensor, coordinates: list, expand_coeff=3.0):
        '''
            picture: torch.Tensor
            coordinates: "indexed set" of 4 coordinates: [x1, y1, x2, y2]
        '''
        x = coordinates[0]
        y = coordinates[1]
        w = coordinates[2] - coordinates[0]
        h = coordinates[3] - coordinates[1]
        w_expanded = int(w * expand_coeff)
        h_expanded = int(h * expand_coeff)
        x_expanded = int(x - (w_expanded - w) / 2.0)
        y_expanded = int(y - (h_expanded - h) / 2.0)

        return torchvision.transforms.functional.crop(
            picture, 
            int(y_expanded), 
            int(x_expanded), 
            int(h_expanded), 
            int(w_expanded),
        )

    def __check_intersection(self, cr_1, cr_2):
        x1 = [cr_1[0], cr_1[2]]
        y1 = [cr_1[1], cr_1[3]]
        x2 = [cr_2[0], cr_2[2]]
        y2 = [cr_2[1], cr_2[3]]

        return not (max(x1) < min(x2) or 
                    min(x1) > max(x2) or 
                    max(y1) < min(y2) or 
                    min(y1) > max(y2))


    def __drop_uninformative_boxes(
                                    self, 
                                    boxes, 
                                    labels, 
                                    probabilities, 
                                    dog_label, 
                                    THRESHOLD_SIZE=25, 
                                    THRESHOLD_PROBABILITY=0.08
        ):
        '''
            drop small useless boxes with low probabilities
        '''
        bad_boxes = np.zeros(len(boxes))
        for i in range(len(boxes)):
            if min(boxes[i][2] - boxes[i][0], boxes[i][3] - boxes[i][1]) <= THRESHOLD_SIZE or \
            (probabilities[i] < THRESHOLD_PROBABILITY and labels[i] != dog_label):
                bad_boxes[i] = 1

        for i in range(len(boxes)):
            if bad_boxes[i] == 1:
                    continue
            for j in range(i + 1, len(boxes)):
                if bad_boxes[j] == 1:
                    continue
                    
                if self.__check_intersection(boxes[i], boxes[j]):
                    if labels[i] == dog_label and labels[j] == dog_label:
                        s1 = (boxes[i][2] - boxes[i][0]) * (boxes[i][3] - boxes[i][1])
                        s2 = (boxes[j][2] - boxes[j][0]) * (boxes[j][3] - boxes[j][1])

                        if s1 > s2:
                            bad_boxes[j] = 1
                        else:
                            bad_boxes[i] = 1
                            break


        result_boxes = []
        result_labels = []

        for i in range(len(bad_boxes)):
            if bad_boxes[i] == 0:
                result_boxes.append(boxes[i])
                result_labels.append(labels[i].cpu().item())

        return result_boxes, result_labels

    def __extract_text(self, path):
        image_cv = cv2.imread(path)
        image_cv = image_cv[:image_cv.shape[0] // 10, :image_cv.shape[0] // 3, :]
        image_cv = cv2.resize(image_cv, (image_cv.shape[1] * 3, image_cv.shape[0] * 3))
        string = pytesseract.image_to_string(image_cv, lang='eng+rus')
        id = re.findall(r"\s.*_\d.*\s", string, re.M)
        if len(id) > 0:
            id = id[0][1:-1]
        else:
            id = None
        date = re.findall(r'^.*\d\d:\d\d:\d\d$', string, re.M)
        if len(date) > 0:
            date = date[0]
        else:
            date = None
        address = re.findall(r'^.*\sдом.*$', string, re.M)
        if len(address) > 0:
            address = address[0]
        else:
            address = None

        return id, address, date

    def __has_owner(self, dog, persons, THRESHOLD=4):
        dist = np.zeros(len(persons), dtype=np.float32)
        for i in range(len(persons)):
                delta_x = abs((persons[i][2] + persons[i][0]) / 2 -  
                                (dog[2] + dog[0]) / 2) / ((dog[2] - dog[0]))

                delta_y = abs((persons[i][3] + persons[i][1]) / 2 -  
                                (dog[3] + dog[1]) / 2) / (persons[i][3] - persons[i][1])

                dist[i] = max(delta_x, delta_y)
        return np.sum(dist < THRESHOLD) > 0

    def __is_an_animal(self, image) -> bool:
        logits_per_image, logits_per_text = model(image, self.tokens_animal)
        return logits_per_image[0].argmax().item() == self.is_animal_index

    def __is_a_dog(self, image) -> bool:
        logits_per_image, logits_per_text = model(image, self.tokens_is_dog)
        return logits_per_image[0].argmax().item() == self.is_dog_index

    def __get_breed_tail(self, image) -> bool:
        logits_per_image, logits_per_text = model(image, self.tokens_breed)
        breed = logits_per_image[0].argmax().item()
        return self.breeds[breed], self.converter_tail[self.breeds[breed]]

    def __get_color(self, image):
        logits_per_image, logits_per_text = model(image, self.tokens_color)
        return self.converter_color[self.colors[logits_per_image.argmax().item()]]

    def __get_features(self, image, box, persons, result_of_extract_text, path):
        is_the_owner_there = int(self.__has_owner(box, persons))
        color = self.__get_color(image)
        breed, tail = self.__get_breed_tail(image)
        cam_id, address, date = result_of_extract_text
        return ResultType(
            os.path.basename(path), 1, 1, 
            is_the_owner_there, 
            color, breed, 
            tail, cam_id, 
            address, date
        )

    def __not_a_dog(self, result_of_extract_text, path):
        cam_id, address, date = result_of_extract_text
        return ResultType(os.path.basename(path), 1, 0, 0, 0, 0, 0, address, cam_id, date)

    def __empty(self, result_of_extract_text, path):
        cam_id, address, date = result_of_extract_text
        return ResultType(os.path.basename(path), 0, 0, 0, 0, 0, 0, address, cam_id, date)

    def __choose_answer(self, results: list) -> ResultType:
        for result in results:
            if result.res["is_it_a_dog"] == 1:
                return result
        return results[0]

    def __call__(self, path):
        image = Image.open(path)

        features = []
        # if it is a png file
        picture_numpy = np.asarray(image)[:, :, :3]

        picture_torch = torch.from_numpy(picture_numpy).\
        to(dtype=torch.float32, device=self.device).permute(2, 0, 1) / 256
        picture_torch.unsqueeze_(0)

        # get detector predictions
        detector_predictions = self.model_detection(picture_torch)[0]
        boxes = detector_predictions["boxes"]
        labels = detector_predictions["labels"]
        probabilities = detector_predictions["scores"]

        bird_here = (inst_classes.index("bird") in labels)

        # drop uninformative boxes
        boxes, labels = self.__drop_uninformative_boxes(
            boxes, 
            labels, 
            probabilities,
            18,
        )

        result_of_extract_text = self.__extract_text(path)

        persons = [box for box, label in zip(boxes, labels) if label == self.person_label]
        for box, label in zip(boxes, labels):
            # if is in not in interesting_indices
            if not label in self.interesting_indices:
                continue

            cropped_image = self.__get_cropped_image(picture_torch, box)
            cropped_image_not_expanded = self.__get_cropped_image(
                picture_torch, box, 1.5
            )


            cropped_clip = self.preprocess_clip(
                torchvision.transforms.ToPILImage()(cropped_image[0])
            ).unsqueeze(0).to(self.device)

            cropped_clip_not_expanded = self.preprocess_clip(
                torchvision.transforms.ToPILImage()(cropped_image_not_expanded[0])
            ).unsqueeze(0).to(self.device)

            # draw_pic_with_rect(cropped_image_not_expanded, [], [])
            # logits_per_image, logits_per_text = model(cropped_clip_not_expanded, self.tokens_animal)
            # print(logits_per_image)

            # if it is not an animal
            if not self.__is_an_animal(cropped_clip_not_expanded):
                continue

            # get features of a dog
            if self.__is_a_dog(cropped_clip):
                features.append(self.__get_features(
                    cropped_clip, 
                    box, 
                    persons, 
                    result_of_extract_text,
                    path
                ))
            else:
                features.append(self.__not_a_dog(
                    result_of_extract_text,
                    path,
                ))

        if len(features) == 0 and bird_here:
            features.append(self.__not_a_dog(
                result_of_extract_text,
                path,
            ))

        if len(features) == 0:
            features.append(self.__empty(
                result_of_extract_text,
                path,
            ))

        return self.__choose_answer(features)
        # return features


find_lost_animal = FindLostAnimal(model_detection, 
               model, 
               preprocess, 
               None, 
               interensting_indices, 
               animals, 
               0, 
               is_dog, 
               0, 
               list(converter_tail.keys()), 
               list(converter_color.keys()),
               converter_tail,
               converter_color,
               1,
               "cuda",
               )

result = find_lost_animal("/content/Тестовые снимки/Только собаки/ТЕМНЫЙ ДЛИННЫЙ/B111.jpg")
for res in result:
    print(res.res)

TypeError: ignored

In [344]:
path_ = "/content/Тестовые снимки/Хоязин и собака/С60.jpg"
result = find_lost_animal(path_)
print(result.res)
# for res in result:
#     print(res.res)

{'filename': 'С60.jpg', 'is_animal_there': 1, 'is_it_a_dog': 1, 'is_the_owner_there': 1, 'color': 1, 'breed': 'a labrador', 'tail': 2, 'address': "P'¥N_hd_TSAO_3647_1", 'cam_id': None, 'date': None}


In [192]:
import pandas as pd

In [321]:
empty_result = []
path_to_dir = "/content/Тестовые снимки/Пустые"
for image_name in tqdm(os.listdir(path_to_dir)):
    path_to_pic = os.path.join(path_to_dir, image_name)
    empty_result.append(
        find_lost_animal(path_to_pic).res
    )

100%|██████████| 48/48 [01:39<00:00,  2.08s/it]


In [ ]:
print(*empty_result, sep="\n")

In [323]:
_cnt = 0
for res in empty_result:
    _cnt += res["is_animal_there"]

print(_cnt / len(empty_result))

0.16666666666666666


In [324]:
empty_result_copy = deepcopy(empty_result)

In [326]:
only_dogs = []

In [349]:
path_to_dir = "/content/Тестовые снимки/Только собаки"
for directory in os.listdir(path_to_dir):
    for image_name in tqdm(os.listdir(os.path.join(path_to_dir, directory))):
        path_to_pic = os.path.join(path_to_dir, directory, image_name)
        try:
            only_dogs.append(
                find_lost_animal(path_to_pic).res
            )
        except Exception:
            print("fucking fuck!!!")

 14%|█▍        | 4/29 [00:11<01:06,  2.66s/it]

fucking fuck!!!


 66%|██████▌   | 19/29 [00:43<00:29,  2.96s/it]

fucking fuck!!!


 19%|█▉        | 7/37 [00:16<01:04,  2.15s/it]

...

  7%|▋         | 2/30 [00:04<01:01,  2.19s/it]

fucking fuck!!!


 83%|████████▎ | 25/30 [00:53<00:09,  1.93s/it]

fucking fuck!!!


100%|██████████| 31/31 [01:17<00:00,  2.50s/it]


In [353]:
only_dogs_copy = deepcopy(only_dogs)

In [350]:
print(*only_dogs, sep="\n")

{'filename': 'А98.jpg', 'is_animal_there': 0, 'is_it_a_dog': 0, 'is_the_owner_there': 0, 'color': 0, 'breed': 0, 'tail': 0, 'address': None, 'cam_id': None, 'date': None}
{'filename': 'А30.jpg', 'is_animal_there': 1, 'is_it_a_dog': 0, 'is_the_owner_there': 0, 'color': 0, 'breed': 0, 'tail': 0, 'address': None, 'cam_id': 'ЦМЧ_ЗАО_027_8', 'date': '27 сент. 2021, 08:45:49'}
{'filename': 'А49.jpg', 'is_animal_there': 1, 'is_it_a_dog': 1, 'is_the_owner_there': 0, 'color': 1, 'breed': 'a labrador', 'tail': 2, 'address': None, 'cam_id': None, 'date': None}
{'filename': 'B19.jpg', 'is_animal_there': 1, 'is_it_a_dog': 1, 'is_the_owner_there': 0, 'color': 1, 'breed': 'a labrador', 'tail': 2, 'address': None, 'cam_id': None, 'date': None}
{'filename': 'B111.jpg', 'is_animal_there': 0, 'is_it_a_dog': 0, 'is_the_owner_there': 0, 'color': 0, 'breed': 0, 'tail': 0, 'address': None, 'cam_id': None, 'date': None}
{'filename': 'B47.jpg', 'is_animal_there': 0, 'is_it_a_dog': 0, 'is_the_owner_there': 0, '

In [352]:
сколько_догов = 0
for дог in only_dogs:
    сколько_догов += дог["is_it_a_dog"]
print(сколько_догов / len(only_dogs))

0.47953216374269003


In [345]:
owner_and_dog = []
path_to_dir = "/content/Тестовые снимки/Хоязин и собака"
for image_name in tqdm(os.listdir(path_to_dir)):
    path_to_pic = os.path.join(path_to_dir, image_name)
    try:
        owner_and_dog.append(
            find_lost_animal(path_to_pic).res
        )
    except Exception:
        print("fucking fuck!!!")

  9%|▉         | 6/67 [00:19<03:05,  3.05s/it]

fucking fuck!!!


 15%|█▍        | 10/67 [00:28<02:42,  2.85s/it]

fucking fuck!!!


 42%|████▏     | 28/67 [01:17<02:13,  3.43s/it]

...

 72%|███████▏  | 48/67 [02:13<01:01,  3.25s/it]

fucking fuck!!!


 93%|█████████▎| 62/67 [02:53<00:18,  3.71s/it]

fucking fuck!!!


100%|██████████| 67/67 [03:01<00:00,  2.71s/it]


In [346]:
print(*owner_and_dog, sep="\n")

{'filename': 'А61.jpg', 'is_animal_there': 1, 'is_it_a_dog': 1, 'is_the_owner_there': 1, 'color': 1, 'breed': 'a labrador', 'tail': 2, 'address': 'РУМ_НЯ 7АС_9544_4', 'cam_id': 'ЗАО улица Крылатские Холмы, дом 32, корпус', 'date': '28 сент. 2021, 07:45:09'}
{'filename': 'А15.jpg', 'is_animal_there': 1, 'is_it_a_dog': 1, 'is_the_owner_there': 1, 'color': 2, 'breed': 'a bull terrier', 'tail': 2, 'address': 'УММ_$АО_027_2', 'cam_id': 'САО Ленинградский проспект, дом 35', 'date': '27 сент. 2021, 08:53:48'}
{'filename': 'B58.jpg', 'is_animal_there': 1, 'is_it_a_dog': 1, 'is_the_owner_there': 1, 'color': 2, 'breed': 'a borzoi', 'tail': 2, 'address': 'PVN_hd_TSAO_5300_3', 'cam_id': 'ЦАО Верхняя Сыромятническая улица, дом 2', 'date': '28 сент. 2021, 07:24:20'}
{'filename': 'С26.jpg', 'is_animal_there': 1, 'is_it_a_dog': 0, 'is_the_owner_there': 0, 'color': 0, 'breed': 0, 'tail': 0, 'address': 'ЦАО Центросоюзный переулок, дом 4', 'cam_id': 'PVN_hd_TSAO_3647_1', 'date': '29 сент. 2021, 05:35:00'

In [341]:
owner_and_dog_copy = deepcopy(owner_and_dog)

In [354]:
not_dogs = []
path_to_dir = "/content/Тестовые снимки/не собаки"
for image_name in tqdm(os.listdir(path_to_dir)):
    path_to_pic = os.path.join(path_to_dir, image_name)
    try:
        not_dogs.append(
            find_lost_animal(path_to_pic).res
        )
    except Exception:
        print("fucking fuck!!!")

100%|██████████| 33/33 [01:19<00:00,  2.41s/it]


In [355]:
print(*not_dogs, sep="\n")

{'filename': 'B109.jpg', 'is_animal_there': 1, 'is_it_a_dog': 1, 'is_the_owner_there': 0, 'color': 1, 'breed': 'a border collie', 'tail': 2, 'address': 'PVN_hd_TSAO_5300_3', 'cam_id': 'ЦАО Верхняя Сыромятническая улица, дом 2', 'date': '26 сент. 2021, 15:44:28'}
{'filename': 'С39.jpg', 'is_animal_there': 1, 'is_it_a_dog': 1, 'is_the_owner_there': 0, 'color': 3, 'breed': 'a dachshund', 'tail': 1, 'address': 'P¥N_hd_UAG_7270_3', 'cam_id': 'ЮАО 1-й Нижний Михайловский проезд, дом |', 'date': '29 Е 2021, 06:49:46'}
{'filename': 'B79.jpg', 'is_animal_there': 1, 'is_it_a_dog': 0, 'is_the_owner_there': 0, 'color': 0, 'breed': 0, 'tail': 0, 'address': 'СВАО проспект Мира, дом 182, корпус 3', 'cam_id': "P'¥N_hd_SVAO_3055_1", 'date': '05 окт. 2021, 08:56:58'}
{'filename': 'B73.jpg', 'is_animal_there': 0, 'is_it_a_dog': 0, 'is_the_owner_there': 0, 'color': 0, 'breed': 0, 'tail': 0, 'address': 'ЦАО Верхняя Сыромятническая улица, дом 2', 'cam_id': 'PVN_hd_TSAO_5300_3', 'date': '28 сент. 2021, 10:08

# Делаем КСВ без пород

In [367]:
order = ['filename', 'is_animal_there', 'is_it_a_dog', 'is_the_owner_there', 'color', "breed", 'tail', 'address', 'cam_id', "date"]
result_array = []
for res in not_dogs:
    result_array.append(
        [res[key] for key in order]
    )

for res in owner_and_dog:
    result_array.append(
        [res[key] for key in order]
    )

for res in only_dogs:
    result_array.append(
        [res[key] for key in order]
    )

for res in empty_result:
    result_array.append(
        [res[key] for key in order]
    )

# not_dogs_array = [list(res.values()) for res in not_dogs]

In [368]:
result_csv = pd.DataFrame(result_array, columns=order)

In [369]:
result_csv.to_csv("result_table_with_breed.csv", index=False)